In [ ]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
import matplotlib.pyplot as plt
from numpy_financial import pmt
%matplotlib inline

In [ ]:
vres = ['Solar PV', 'Wind onshore', 'Wind offshore']

In [ ]:
colors = {
    'Solar PV':      (255/255, 190/255,   0/255), 
    'Wind onshore':  ( 70/255, 140/255, 210/255), 
    'Wind offshore': (  0/255, 100/255, 170/255), 
}

# Input parameters
Base case:

In [ ]:
p_base = {
                     # value, unit
    # General
    'Interest rate':  (    7, '\%/a'             ),
    'H2 price':       (    2, '€/kg_{H2}'        ),
    'H2 LHV':         ( 33.3, 'kWh_{H2}/kg_{H2}' ),
    'El supplement':  (    0, '€/MWh_{el}'       ),
     
    # Electrolyzer
    'Electrolyzer CAPEX':        (  450, '€/kW_{el}'        ),
    'Electrolyzer OPEX fix':     (    2, '% of CAPEX'       ),
    'Electrolyzer OPEX var':     (  0.1, '€/kg_{H2}'        ),
    'Electrolyzer lifetime':     (   25, 'a'                ),
    'Electrolyzer efficiency':   (   21, 'kg_{H2}/MWh_{el}' )    
}

Sensitivity:

In [ ]:
p_sensitivity = {
                     #  min,  max,  step     units 
    # General
    'Interest rate':  (   4,   10,    3),  # %/a
    'H2 price':       ( 1.5,  2.5, 0.25),  # €/kg_H2
    'El supplement':  (   0,    0,    0),  # €/MWh_el
    
    # Electrolyzer
    'Electrolyzer CAPEX':        ( 100,  800, 175),  # €/kW_el
    'Electrolyzer lifetime':     (  20,   30,   5),  # a
    'Electrolyzer efficiency':   (  20,   22,   1)   # kg_H2/kWh_el
}

# Time series

This article uses renewable profiles from the METIS dataset, which can be downloaded at https://ec.europa.eu/energy/data-analysis/energy-modelling/metis/metis-scripts-and-data_en. To rund this script, copy the country-wise CSV files for solar, wind onshore, and wind offshore into a subfolder named "input/":

In [ ]:
countries_offshore = {
    'BE', 'BG', 'CY', 'DE', 'DK', 'EE', 'ES', 
    'FI', 'FR', 'GB', 'GR', 'HR', 'IE', 'IT', 
    'LT', 'LV', 'MT', 'NL', 'PL', 'PT', 'SE'
}
countries_PV = countries_offshore | {
    'AT', 'CH', 'CZ', 'HU', 'LU', 'ME', 'MK', 
    'RO', 'RS', 'SI', 'SK'
}
countries_onshore = countries_PV | {
    'BA', 'NO'
}

In [ ]:
ts = {
    'Solar PV': pd.concat(
        [pd.read_csv(
            'input/Solar_fleet_{}__availability.csv'.format(country), 
            index_col=0, sep=';', decimal=',', header=1
        ) for country in countries_PV],
        axis=1
    ),
    'Wind onshore': pd.concat(
        [pd.read_csv(
            'input/Wind_onshore_fleet_{}__availability.csv'.format(country), 
            index_col=0, sep=';', decimal=',', header=1
        ) for country in countries_onshore],
        axis=1
    ),
    'Wind offshore': pd.concat(
        [pd.read_csv(
            'input/Wind_offshore_fleet_{}__availability.csv'.format(country), 
            index_col=0, sep=';', decimal=',', header=1
        ) for country in countries_offshore],
        axis=1
    ),
}

# Sort in descending order
ts = {
    vre: pd.DataFrame(np.sort(ts[vre].values, axis=0)[::-1])
    for vre in vres
}

Visualization:

In [ ]:
ts_median = {
    vre: ts[vre].median(axis=1)
    for vre in vres
}

In [ ]:
ts_quantiles = {
    vre: {
        'q5': ts[vre].quantile(.05, axis=1),
        'q95': ts[vre].quantile(.95, axis=1)
    }
    for vre in vres
}

In [ ]:
def profiles(ax, vre):
    
    # Plot 5-95% range of profiles
    ax.fill_between(
        ts_quantiles[vre]['q5'].index[::-1], ts_quantiles[vre]['q5'], ts_quantiles[vre]['q95'],
        color=colors[vre], alpha=.5
    )
    
    # Plot median profile
    ax.plot(
        ts_median[vre].index[::-1], ts_median[vre], 
        color=colors[vre]
    )
    
    # Vertical line and textbox
    a1000 = ts_median[vre].iloc[:1000].mean()
    ax.axvline(7660, color='k', linewidth=.5, linestyle='dashed')
    ax.plot(pd.Series({7660: a1000, 8760: a1000}), color=colors[vre])
    ax.text(
        5660, a1000 - (.05 if vre=='Wind offshore' else .02),
        r'$\alpha$: '+'{}'.format(round(a1000,2)), 
        bbox=dict(facecolor='white', alpha=1,
                  ec=colors[vre])
    )

    # X axis
    ax.set_xlabel('Hours, sorted by capacity factor')
    ax.set_xlim(0, 8760)
    
    # Y axis
    ax.set_ylabel('Capacity factor')
    ax.set_ylim(0, 1)
    
    # Title
    ax.set_title(vre)
    
    # Grid
    plt.grid(axis='y', zorder=0, color='grey', linewidth=.5)
    
    return ax

In [ ]:
fig = plt.figure(figsize=(10,3.5))

ax = {}
for i, vre in enumerate(vres):
    ax[i] = plt.subplot(131+i)
    ax[i] = profiles(ax[i], vre)
for i in range(1,3):
    ax[i].set_ylabel('')
    ax[i].set_yticklabels([])

plt.tight_layout()
plt.savefig('graphs/Profiles - VRE.png', dpi=300)

In [ ]:
# Calculate full load hours
df = pd.DataFrame()
df['Full load hours'] = pd.concat(
    [ts[vre].sum()
     for vre in vres],
    keys = vres
)

# Copy vre and values of the fixed parameter to columns
df['VRE'] = df.index.get_level_values(0)
df.head()

df['x'] = df['VRE'].apply(lambda x: {
    'Solar PV': 1,
    'Wind onshore': 1,
    'Wind offshore': 1
}[x])

# Boxplot
ax = sns.boxplot(
    data=df, x='x', y='Full load hours', hue='VRE', 
    palette=colors.values(), saturation=1, whis=[5,95],                 
)

# X axis
plt.xticks([-.265, 0, .265], vres)
plt.xlabel('')

# Grid and no legend
plt.grid(axis='y', zorder=0, color='grey', linewidth=.5)
plt.gca().legend_.remove()

# Functions

In [ ]:
def dispatch_price(p):
    
    # Dispatch electricity price in €/MWh_el
    dispatch_price = (p['H2 price'] - p['Electrolyzer OPEX var']) * p['Electrolyzer efficiency'] - p['El supplement']
    
    return dispatch_price

In [ ]:
def zero_prices_max(p):
    
    # Annualized fixed cost of electrolysis in €/kW_el
    afc = - pmt(p['Interest rate']/100, p['Electrolyzer lifetime'], p['Electrolyzer CAPEX']) \
          + p['Electrolyzer OPEX fix'] * p['Electrolyzer CAPEX'] / 100
    
    # Maximum frequency of zero prices in h
    h_zero_max = afc / dispatch_price(p) * 1000
    
    return h_zero_max

In [ ]:
def market_value_min(p, ts):
    
    # Minimal market value in €/MWh_el
    mv_min = ts.iloc[int(zero_prices_max(p)):].sum() / ts.sum() * dispatch_price(p)
    
    return mv_min

In [ ]:
def monte_carlo(vre, p_fixed={}, ts=ts):
    
    # Take base parameters
    p = {key: value[0] for key, value in p_base.items()}
    
    mv = []
    for i in range(n):
        
        # Random choice of VRE profile
        tsx = ts[vre].iloc[:,np.random.choice(ts[vre].shape[1])]
        
        # Uniformly-distributed random values in sensitivity range (a, b)
        for key, (a, b, step) in p_sensitivity.items():
            p[key] = a + np.random.random() * (b-a)
        
        # Override values with constants
        for key, c in p_fixed.items():
            p[key] = c
        
        mv.append(market_value_min(p, tsx))
        
    return pd.Series(mv)

# Analysis

## Density functions

In [ ]:
n = 1000000

In [ ]:
mv = {}
for vre in vres:
    t = time.time()
    mv[vre] = monte_carlo(vre)
    print(time.time() - t)

In [ ]:
def density(vre):
    
    mean = round(mv[vre].mean(),1)
    std = round(mv[vre].std(),1)
    
    median = round(mv[vre].median(),1)
    q5 = round(mv[vre].quantile(.05),1)
    q95 = round(mv[vre].quantile(.95),1)

    # Density plot
    ax = sns.distplot(mv[vre], hist=True, kde=True, color=colors[vre])

    # Vertical line and textbox
    ax.axvline(median, color=colors[vre])
    ax.text(
        median+2, 0.051, 
        'mean: {}\nstd: {}'.format(mean, std), 
        bbox=dict(facecolor='white', alpha=1,
                  ec=colors[vre])
    )

    # X axis
    ax.set_xlabel('Minimum market value (€/MWh)')
    ax.set_xlim(0, 50)
    
    # Y axis
    ax.set_ylabel('Density (%)')
    ax.set_ylim(0, 0.06)
    ax.set_yticklabels(range(0,7,1))
    
    # Title
    ax.set_title(vre)
    
    return ax

In [ ]:
# This creates one figure per VRE
single = True
if single:
    for vre in vres:
        fig = plt.figure()
        density(vre)
        plt.savefig('graphs/Density - {}.png'.format(vre), dpi=300)

In [ ]:
# This creates one figure with all VRE
multiple = True
if multiple:
    
    fig = plt.figure(figsize=(10,3.5))

    ax = {}
    for i, vre in enumerate(vres):
        ax[i] = plt.subplot(131+i)
        ax[i] = density(vre)
    for i in range(1,3):
        ax[i].set_ylabel('')
        ax[i].set_yticklabels([])

    plt.tight_layout()
    plt.savefig('graphs/Density - VRE.png', dpi=300)

## Sensitivity anaylsis

In [ ]:
n=100000

In [ ]:
p_list = ['H2 price', 'Electrolyzer CAPEX', 'El supplement']

In [ ]:
mv_fixed = {}
for p_fixed in p_list:
    
    # Define electricity price supplement only for dedicated sensitivity
    if p_fixed == 'El supplement':
        p_sensitivity['El supplement'] = (0, 20, 5)  # €/MWh_el
    
    # Define range for the parameter that is fixed
    a, b, step = p_sensitivity[p_fixed]
    rng = np.arange(a, b+step, step)
    
    # Create a dataframe with sensitivity results
    df = pd.DataFrame()
    df['Market value'] = pd.concat(
        [pd.concat(
             [monte_carlo(vre, {p_fixed: x}) for x in rng], keys=rng
         ).clip(lower=0)
         for vre in vres],
        keys = vres
    )
    
    # Copy vre and values of the fixed parameter to columns
    df['VRE'] = df.index.get_level_values(0)
    df[p_fixed] = df.index.get_level_values(1)
    
    # Reset electricity price supplement
    p_sensitivity['El supplement'] = (0, 0, 0)  # €/MWh_el
    
    mv_fixed[p_fixed] = df

Key parameters:

In [ ]:
handles = None
labels = None
def sensitivity(p_fixed, legend=True):
    
    # Grid
    plt.grid(axis='y', zorder=0, color='grey', linewidth=.5)
    
    # Boxplot
    ax = sns.boxplot(data=mv_fixed[p_fixed], x=p_fixed, y='Market value', hue='VRE', whis=[5,95], 
                     palette=colors.values(), saturation=1, showfliers=False)
    
    # X axis
    ax.set_xlabel(p_fixed + ' ($\mathregular{' + p_base[p_fixed][1] + '}$)')
    
    # Y axis
    ax.set_ylim(0, 50)
    ax.set_ylabel('Minimum market value (€/MWh)')
    
    # Legend
    if legend:
        legend = ax.legend(ncol=3, bbox_to_anchor=(0.5, 1), loc='lower center')
    else:
        ax.legend([], [], bbox_to_anchor=(0.5, 1), loc='lower center')
    
    # Safe handles and labels for FLH sensitivity
    global handles, labels
    handles, labels = ax.get_legend_handles_labels()
    
    return ax

In [ ]:
# This creates one figure per parameter
for p_fixed in p_list:
    fig = plt.figure()
    sensitivity(p_fixed)
    plt.savefig('graphs/Sensitivity - {}.png'.format(p_fixed), dpi=300)

In [ ]:
# This creates one figure with all parameters
multiple = True
if multiple:
    
    fig = plt.figure(figsize=(10,3.5))

    ax = {}
    for i, p_fixed in enumerate(p_list):
        ax[i] = plt.subplot(131+i)
        if i == 1:
            ax[i] = sensitivity(p_fixed, legend=False)
        else:
            ax[i] = sensitivity(p_fixed, legend=False)
    for i in range(1,3):
        ax[i].set_ylabel('')
        ax[i].set_yticklabels([])

    plt.tight_layout()
    
    ax[0].get_legend().remove()
    ax[1].legend(ncol=3, bbox_to_anchor=(0.5, 1), loc='lower center')
    ax[2].get_legend().remove()
    
    plt.savefig('graphs/Sensitivities.png', dpi=300)

Other parameters:

In [ ]:
p_list = ['Interest rate', 'Electrolyzer lifetime', 'Electrolyzer efficiency']

In [ ]:
mv_fixed = {}
for p_fixed in p_list:
    
    # Define electricity price supplement only for dedicated sensitivity
    if p_fixed == 'El supplement':
        p_sensitivity['El supplement'] = (0, 20, 5)  # €/MWh_el
    
    # Define range for the parameter that is fixed
    a, b, step = p_sensitivity[p_fixed]
    rng = np.arange(a, b+step, step)
    
    # Create a dataframe with sensitivity results
    df = pd.DataFrame()
    df['Market value'] = pd.concat(
        [pd.concat(
             [monte_carlo(vre, {p_fixed: x}) for x in rng], keys=rng
         ).clip(lower=0)
         for vre in vres],
        keys = vres
    )
    
    # Copy vre and values of the fixed parameter to columns
    df['VRE'] = df.index.get_level_values(0)
    df[p_fixed] = df.index.get_level_values(1)
    
    # Reset electricity price supplement
    p_sensitivity['El supplement'] = (0, 0, 0)  # €/MWh_el
    
    mv_fixed[p_fixed] = df

In [ ]:
# This creates one figure per parameter
for p_fixed in p_list:
    fig = plt.figure()
    sensitivity(p_fixed)
    plt.savefig('graphs/Sensitivity - {}.png'.format(p_fixed), dpi=300)

In [ ]:
# This creates one figure with all parameters
multiple = True
if multiple:
    
    fig = plt.figure(figsize=(10,3.5))

    ax = {}
    for i, p_fixed in enumerate(p_list):
        ax[i] = plt.subplot(131+i)
        if i == 1:
            ax[i] = sensitivity(p_fixed, legend=False)
        else:
            ax[i] = sensitivity(p_fixed, legend=False)
    for i in range(1,3):
        ax[i].set_ylabel('')
        ax[i].set_yticklabels([])

    plt.tight_layout()
    
    ax[0].get_legend().remove()
    ax[1].legend(ncol=3, bbox_to_anchor=(0.5, 1), loc='lower center')
    ax[2].get_legend().remove()
    
    plt.savefig('graphs/Sensitivities - other.png', dpi=300)

VRE full load hours:

In [ ]:
def sensitivity_flh():

    p_fixed = 'Full load hours'
    
    # Ranges
    rngs = {
        'Solar PV': range(900, 1301, 300),
        'Wind onshore': range(1600, 2501, 300),
        'Wind offshore': range(2800, 4301, 300),
    }
    b = 100
    
    # Create a dataframe with sensitivity results
    df = pd.DataFrame()
    sers = []
    keys = []
    for vre in vres:
        for x in rngs[vre]:
            ts_filtered = {
                vre: ts[vre].loc[:, (ts[vre].sum() >= x-b) & (ts[vre].sum() <= x+b)]
            }
            sers.append(monte_carlo(vre, ts=ts_filtered))
            keys.append(x)
    df['Market value'] = pd.concat(
        sers, keys=keys
    )#.clip(lower=0)
    
    # Copy vre and values of the fixed parameter to columns
    df[p_fixed] = df.index.get_level_values(0)
    
    # Grid
    plt.grid(axis='y', zorder=0, color='grey', linewidth=.5)
    
    # Boxplot
    ax = sns.boxplot(data=df, x=p_fixed, y='Market value', saturation=1, whis=[5, 95], showfliers=False,
                     palette=2*[colors['Solar PV']] + 4*[colors['Wind onshore']] + 6*[colors['Wind offshore']])
    
    # X axis
    plt.xticks(rotation=90)
    ax.set_xlabel('Full load hours (h/a)')
    
    # Y axis
    ax.set_ylim(0, 50)
    ax.set_ylabel('Minimum market value (€/MWh)')
    
    # Legend
    ax.legend(handles, labels, ncol=3, bbox_to_anchor=(0.5, 1), loc='lower center')
    
    # Safe figure
    plt.tight_layout()
    plt.savefig('graphs/Sensitivity - {}.png'.format(p_fixed), dpi=300)

In [ ]:
sensitivity_flh()